In [ ]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB

In [ ]:
#Load Data
df = pd.read_csv("iris.csv")
# df = df.drop(axis = 1)
print(df)

#Train Test Split
train = df.sample(frac = 0.7, random_state = 1)
test = df.drop(train.index)

y_train = train["species"]
x_train = train.drop("species", axis = 1)

y_test = test["species"]
x_test = test.drop("species", axis = 1)

#Training – Count Posterior
means = train.groupby(["species"]).mean() # Find mean of each class
var = train.groupby(["species"]).var() # Find variance of each class
prior = (train.groupby("species").count() / len(train)).iloc[:,1] # Find prior probability of each class
classes = np.unique(train["species"].tolist()) # Storing all possible classes


In [ ]:
#Classification
def Normal(n, mu, var):
    # Function to return pdf of Normal(mu, var) evaluated at x
    sd = np.sqrt(var)
    pdf = (np.e ** (-0.5 * ((n - mu)/sd) ** 2)) / (sd * np.sqrt(2 * np.pi))
    return pdf


In [ ]:
def Predict(X):
    Predictions = []
    
    for i in X.index: # Loop through each instances
        ClassLikelihood = []
        instance = X.loc[i]
        for cls in classes: # Loop through each class
            FeatureLikelihoods = []
            FeatureLikelihoods.append(np.log(prior[cls])) # Append log prior of class 'cls'
            for col in x_train.columns: # Loop through each feature
                data = instance[col]
                mean = means[col].loc[cls] # Find the mean of column 'col' that are in class 'cls'
                variance = var[col].loc[cls] # Find the variance of column 'col' that are in class 'cls'
                Likelihood = Normal(data, mean, variance)
                if Likelihood != 0:
                    Likelihood = np.log(Likelihood) # Find the log-likelihood evaluated at x
                else:
                    Likelihood = 1/len(train) 
                
                FeatureLikelihoods.append(Likelihood)
                
            TotalLikelihood = sum(FeatureLikelihoods) # Calculate posterior
            ClassLikelihood.append(TotalLikelihood)
            
        MaxIndex = ClassLikelihood.index(max(ClassLikelihood)) # Find largest posterior position
        Prediction = classes[MaxIndex]
        Predictions.append(Prediction)     
    return Predictions


In [ ]:
def Accuracy(y, prediction):
    # Function to calculate accuracy
    y = list(y)
    prediction = list(prediction)
    score = 0
    for i, j in zip(y, prediction):
        if i == j:
            score += 1   
    return score / len(y)


In [ ]:
PredictTrain = Predict(x_train)
PredictTest = Predict(x_test)

print('Training Accuracy: %.4f' % round(Accuracy(y_train, PredictTrain), 5))
print('Testing Accuracy: %.4f' % round(Accuracy(y_test, PredictTest), 5))


In [ ]:
clf = GaussianNB()
clf.fit(x_train, y_train)
SkTrain = clf.predict(x_train) # Predicting on the train set
SkTest = clf.predict(x_test) # Predicting on the test set

print('Training Accuracy: %.4f' % round(Accuracy(y_train, SkTrain), 5))
print('Testing Accuracy: %.4f' % round(Accuracy(y_test, SkTest), 5))
